# TCC - DSA T212 - Prova de Conceito

## **Buscando os dados na Portal Dados Abertos** [Dados Abertos gov.br](http://dados.gov.br)

| Ano  | Endereço                  |Ano  | Endereço                  | Ano  | Endereço                  |Ano  | Endereço                  |
|------|-------------------------- |------|-------------------------- |------|-------------------------- |------|-------------------------- |
|2002| [DOU 2002](http://dados.gov.br/dataset/diario-oficial-da-uniao-materias-publicadas-em-2002)|   2003  | [DOU 2003](http://dados.gov.br/dataset/diario-oficial-da-uniao-materias-publicadas-em-2003)|   2004  | [DOU 2004](http://dados.gov.br/dataset/diario-oficial-da-uniao-materias-publicadas-em-2004)|   2005  | [DOU 2005](http://dados.gov.br/dataset/diario-oficial-da-uniao-materias-publicadas-em-2005)|
|   2006  | [DOU 2006](http://dados.gov.br/dataset/diario-oficial-da-uniao-materias-publicadas-em-2006)|   2007  | [DOU 2007](http://dados.gov.br/dataset/diario-oficial-da-uniao-materias-publicadas-em-2007)|   2008  | [DOU 2008](http://dados.gov.br/dataset/diario-oficial-da-uniao-materias-publicadas-em-2008)|   2009  | [DOU 2009](http://dados.gov.br/dataset/diario-oficial-da-uniao-materias-publicadas-em-2009)|
|   2010  | [DOU 2010](http://dados.gov.br/dataset/diario-oficial-da-uniao-materias-publicadas-em-2010)|   2011  | [DOU 2011](http://dados.gov.br/dataset/diario-oficial-da-uniao-materias-publicadas-em-2011)|   2012  | [DOU 2012](http://dados.gov.br/dataset/diario-oficial-da-uniao-materias-publicadas-em-2012)|   2013  | [DOU 2013](http://dados.gov.br/dataset/diario-oficial-da-uniao-materias-publicadas-em-2013)|
|   2014  | [DOU 2014](http://dados.gov.br/dataset/diario-oficial-da-uniao-materias-publicadas-em-2014)|   2015  | [DOU 2015](http://dados.gov.br/dataset/diario-oficial-da-uniao-materias-publicadas-em-2015)|   2016  | [DOU 2016](http://dados.gov.br/dataset/diario-oficial-da-uniao-materias-publicadas-em-2016)|   2017  | [DOU 2017](http://dados.gov.br/dataset/diario-oficial-da-uniao-materias-publicadas-em-2017)|
|   2018  | [DOU 2018](http://dados.gov.br/dataset/diario-oficial-da-uniao-materias-publicadas-em-2018)|   2019  | [DOU 2019](http://dados.gov.br/dataset/diario-oficial-da-uniao-materias-publicadas-em-2019)|   2020  | [DOU 2020](http://dados.gov.br/dataset/diario-oficial-da-uniao-materias-publicadas-em-2020)|  2021  | [DOU 2021](http://dados.gov.br/dataset/diario-oficial-da-uniao-materias-publicadas-em-2021)|
|  2022  | [DOU 2022](http://dados.gov.br/dataset/diario-oficial-da-uniao-materias-publicadas-em-2022)|

## Parâmetros iniciais do caderno notebook

In [73]:
#configuração da dimensões da tela de apresentação
from notebook.services.config import ConfigManager
cm = ConfigManager()

#defina as dimensões da apresentação.
cm.update('livereveal', {
              'width': 1000,
              'height': 600,
              'scroll': True,
         })
base_path = 'C:/projetos/prova-de-conceito/dados/'
#base_path = '/content/drive/My Drive/Colab Notebooks/'
url_base_dou = "http://dados.gov.br/dataset/diario-oficial-da-uniao-materias-publicadas-em-"


### Importando as bibliotecas


In [75]:
!pip install wget

In [76]:
import os

import pandas as pd
print("Versão pandas: " + pd.__version__) 

import sklearn as sk
print("Versão sklearn: " + sk.__version__) 

import numpy as np
import io
from sklearn.model_selection import train_test_split

import urllib.request as req
print("Versão urllib.request: " +req.__version__)

from bs4 import BeautifulSoup
from io import StringIO
import requests, zipfile
from xml.etree import ElementTree
import os
import csv
from os import listdir
from os.path import isfile, join
import time
from datetime import datetime
import pytz
import re
import math
#import spacy
import wget 

  

Versão pandas: 1.4.4
Versão sklearn: 1.1.2
Versão urllib.request: 3.9


### Funções utilizadas

In [77]:
class Main:     
    def show_current_time(self):
        ini = time.time()
        tz_SP = pytz.timezone('America/Sao_Paulo') 
        now = datetime.now(tz_SP)
        current_time = now.strftime("%d/%m/%Y %H:%M:%S")
        print("Current Time =", current_time)  

### Montando a estrutura de diretórios

In [78]:
try:  
    print(base_path)
    os.chdir(base_path)
    dirname = "dou"
    isdir = os.path.isdir(base_path+dirname) 
    if not isdir:
        print("criando diretório dou em : "+ base_path+dirname)
        os.mkdir(dirname)
    path = base_path+dirname+"/"
    os.chdir(path)
    #print(path)
  
    today = datetime.today()
    dir_years = range(2002,today.year+1)
    for year in dir_years:
        print(path+str(year))
        isdir = os.path.isdir(path+str(year))  
        if not isdir:
          print("criando diretorio do ano " + str(year))
          os.mkdir(path+str(year))
        os.chdir(path+str(year))
        months = ['01','02','03','04','05','06','07','08','09','10','11','12']
        for month in months:
          #print(path+str(year)+"/"+month)
          isdir = os.path.isdir(path+str(year)+"/"+month)  
          if not isdir:
            print("criando diretorio do mes " + month +"/"+str(year))
            os.mkdir(path+str(year)+"/"+month)
          os.chdir(path+str(year)+"/"+month)
          dir_secoes = ['01','02','03']
          for secao in dir_secoes:
            #print(path+str(year)+"/"+month+'/'+secao)
            isdir = os.path.isdir(path+str(year)+"/"+month+'/'+secao)  
            if not isdir:
              print("criando diretorio do mes" + month + " seção " + secao)
              os.mkdir(path+str(year)+"/"+month+'/'+secao)        
except FileExistsError:
      print("Diretório " + base_path + dirname + " já existe.") 
except:
      print("Erro ao criar Diretório " + base_path + dirname +" .")
print("Término da criação da árvore de diretórios.") 

C:/projetos/prova-de-conceito/dados/
C:/projetos/prova-de-conceito/dados/dou/2002
C:/projetos/prova-de-conceito/dados/dou/2003
C:/projetos/prova-de-conceito/dados/dou/2004
C:/projetos/prova-de-conceito/dados/dou/2005
C:/projetos/prova-de-conceito/dados/dou/2006
C:/projetos/prova-de-conceito/dados/dou/2007
C:/projetos/prova-de-conceito/dados/dou/2008
C:/projetos/prova-de-conceito/dados/dou/2009
C:/projetos/prova-de-conceito/dados/dou/2010
C:/projetos/prova-de-conceito/dados/dou/2011
C:/projetos/prova-de-conceito/dados/dou/2012
C:/projetos/prova-de-conceito/dados/dou/2013
C:/projetos/prova-de-conceito/dados/dou/2014
C:/projetos/prova-de-conceito/dados/dou/2015
C:/projetos/prova-de-conceito/dados/dou/2016
C:/projetos/prova-de-conceito/dados/dou/2017
C:/projetos/prova-de-conceito/dados/dou/2018
C:/projetos/prova-de-conceito/dados/dou/2019
C:/projetos/prova-de-conceito/dados/dou/2020
C:/projetos/prova-de-conceito/dados/dou/2021
C:/projetos/prova-de-conceito/dados/dou/2022
Término da criação

### Carregamento dos arquivos .zip com xml da base DOU do portal de dados abertos do governo federal

In [94]:
#years = ['2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015',
#         '2016','2017','2018','2019','2020','2021','2022']

years=['2019','2020','2021','2022']
meses=['01','02','03','04','05','06','07','08','09','10','11','12']
#meses = ['01','02','03','04','05','06','07','08','09','10','11','12']


fx = Main()

fx.show_current_time()


for year in years:
  urlopen =  url_base_dou + year 
  if year == '2017':
    urlopen = 'http://dados.gov.br/dataset/diario-oficial-da-uniao'
  resp = req.urlopen(urlopen)
  conteudo = resp.read(int(resp.getheader('Content-Length')))  
  parsed_html = BeautifulSoup(StringIO(conteudo.decode("UTF-8")))  
  for link in parsed_html.body.findAll('a', attrs={'class':'resource-url-analytics'}):
    url = link.get('href').split("?url=")[1]
    filename = "S"+link.get('href').split('/S')[-1][0:8]
    mes = filename[-6:-4]
    ano = filename[-4:]
    secao = filename[-8:-6]
    path = base_path+"dou/"+year+'/'+mes+'/'+secao+'/'
    file_download = path + filename + '.zip'
    isfile = os.path.isfile(file_download) 
    os.chdir(path) 
    if mes in meses :
        #print(file_download)
        #print(url)
        if not isfile:
            #os.remove(file_download)
            try:    
                wget.download(url)   
                print(' SUCESSO = download zip file to: ' + file_download + ' from url: '+url)
            except:
                print(' ERRO = downloading zip file to: ' + file_download + ' from url: '+url)
        else:
            print(' arquivo já carregado: ' + file_download)
  print("Concluído o download dos arquivos zip do ano :" + year)
  fx.show_current_time()



Current Time = 09/09/2022 10:29:22
 arquivo já carregado: C:/projetos/prova-de-conceito/dados/dou/2019/01/01/S01012019.zip
 arquivo já carregado: C:/projetos/prova-de-conceito/dados/dou/2019/01/02/S02012019.zip
 arquivo já carregado: C:/projetos/prova-de-conceito/dados/dou/2019/01/03/S03012019.zip
 arquivo já carregado: C:/projetos/prova-de-conceito/dados/dou/2019/02/01/S01022019.zip
 arquivo já carregado: C:/projetos/prova-de-conceito/dados/dou/2019/02/02/S02022019.zip
 arquivo já carregado: C:/projetos/prova-de-conceito/dados/dou/2019/02/03/S03022019.zip
 arquivo já carregado: C:/projetos/prova-de-conceito/dados/dou/2019/03/01/S01032019.zip
 arquivo já carregado: C:/projetos/prova-de-conceito/dados/dou/2019/03/02/S02032019.zip
 arquivo já carregado: C:/projetos/prova-de-conceito/dados/dou/2019/03/03/S03032019.zip
 arquivo já carregado: C:/projetos/prova-de-conceito/dados/dou/2019/04/01/S01042019.zip
 arquivo já carregado: C:/projetos/prova-de-conceito/dados/dou/2019/04/02/S02042019.z